# Installing Libraries

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install spacy
!sudo apt update

In [ ]:
!pip install pytesseract
!apt-get install poppler-utils
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 5.7 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (261 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Reading package lists... Done
Building depende

**Note:** Restart Runtime After Installation

# Importing Packages

In [ ]:
import cv2 
import matplotlib.pyplot as plt
import pytesseract
import shutil
import os
import random
import nltk
import numpy as np
import re
try:
 from PIL import Image
except ImportError:
 import Image

# Downloading Packages

In [ ]:
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Pre Processing Image

In [ ]:
def ZoomImage(img, zoom_factor=2):
    return cv2.resize(img, None, fx= zoom_factor, fy= zoom_factor, interpolation= cv2.INTER_LINEAR)

def GrayScaleImage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = 255*(gray < 128).astype(np.uint8) 
    return gray

def InvertImage(img):
    return cv2.bitwise_not(img)

def CropImage(img):
    # Find all non-zero points (text)
    coords = cv2.findNonZero(img) 
    # Find minimum spanning bounding box
    x, y, w, h = cv2.boundingRect(coords) 
    # Crop the image - note we do this on the original image
    img = img[y:y+h, x:x+w] 
    return img

# Creating Bounding Box In Image

In [ ]:
def CreateBoundingBox(img):
    h, w, c = img.shape
    boxes = pytesseract.image_to_boxes(img) 
    for b in boxes.splitlines():
        b = b.split(' ')
        img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)
    return img

# Extracting Text From Image

In [ ]:
def ExtractText(img) : 
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(img, config=custom_config)
    return text

# Extracting Name

Pre Processing For Name

In [ ]:
def text_preprocessing(text):
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  newSentence = ""
  for word in text:
      if (word in punctuations):
          newSentence = newSentence + " "
      else: 
          newSentence = newSentence + word

  return newSentence

def remove_white_spaces(text):
  token_words = word_tokenize(text)
  new_sentence = []
  for word in token_words:
    new_sentence.append(word) 
    new_sentence.append(" ") 
  return "".join(new_sentence)

def removing_extra(text):
  new_sentence = ""
  result = ""
  token_sentence = sent_tokenize(text)
  if "applying for" in token_sentence[0].lower() or "applied for" in token_sentence[0].lower():
    sentences = token_sentence[0].split("\n")
    for sent in sentences:
      if "applying for" in sent.lower() or "applied for" in sent.lower():
        continue
      else:
        new_sentence = new_sentence + sent + "\n"

    i = 0
    for sentence in token_sentence:
      if i == 0:
        result = result + new_sentence + "\n"
      else:
        result = result + sentence  + "\n"
      i += 1
    return result 
  else:
    return text

In [ ]:
import spacy
from spacy.matcher import Matcher
from nltk.tokenize import sent_tokenize, word_tokenize

nlp = spacy.load('en_core_web_sm')

ignore_text1 = "CURRICULUM"
ignore_text2 = "VITAE"
ignore_text3 = "VITA"

matcher = Matcher(nlp.vocab)

def extract_name(content):
    # new_content = text_preprocessing(content)
    new_content = removing_extra(content)
    # print(new_content)
    nlp_text = nlp(new_content)
    punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        # print(span.text)
        if not(ignore_text1.lower() in span.text.lower() or ignore_text2.lower() in span.text.lower() or ignore_text3.lower() in span.text.lower()):
          token_words = word_tokenize(span.text)
          flag = 0
          for word in token_words:
            if word in punctuations:
              flag = 1 
              break
          if flag == 0:
            return span.text
    return ""

# Extracting Phone Number

In [ ]:
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
    if phone:
        number = ''.join(phone[0])
 
        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

## Processing Phone Number

# Extracting Email ID

In [ ]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

## Processing Email ID

In [ ]:
def process_emails(emailID):

  # Identifying the string slicing for gmail
  if type(emailID) == list:
    emailNum = 0
    for eml in emailID:
        for i in range(len(eml)):
          if eml[i] == '@':
            index1 = i
            break
        for j in range(index1 , len(eml)):
          if eml[j] == '.':
            index2 = j
            break
        if eml[index1 + 1:index2] == 'qmail' or eml[index1 + 1:index2] == 'gqmail':
          part1 = eml[0:index1 + 1]
          part2 = eml[index1 + 1 : index2]
          part3 = eml[index2:len(eml)]
          eml = part1 + 'gmail' + part3
        emailID[emailNum] = eml
        emailNum =  emailNum + 1
  else:
    for i in range(len(emailID)):
      if emailID[i] == '@':
        index1 = i
        break
    for j in range(index1 , len(emailID)):
      if emailID[j] == '.':
        index2 = j
        break
    part1 = emailID[0:index1 + 1]
    part2 = emailID[index1 + 1 : index2]
    part3 = emailID[index2:len(emailID)]
    emailID = part1 + 'gmail' + part3

# Main Program

In [ ]:
import csv
import glob

with open('output.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    for i in range(30):
        IMG_PATH = '/content/drive/MyDrive/sampleresumes/' + str(i+110)
        text = ""
        
        # Img Pre Processing
              
        img = cv2.imread(IMG_PATH + '/page0.jpeg' )

        img = ZoomImage(img , 3)
        img = GrayScaleImage(img)
        img = CropImage(img)
        img = InvertImage(img)

        cv2.imwrite(IMG_PATH + '/0.png', img)
        img = cv2.imread(IMG_PATH + '/0.png')

        boundedImg = CreateBoundingBox(img)
        cv2.imwrite(IMG_PATH + '/a.png', boundedImg)
        
        img = cv2.imread(IMG_PATH + '/0.png')

        # NLP Part

        text = ExtractText(img)
        # print(text)      
        personName = extract_name(text.title())
        contactNum = extract_phone_number(text.lower())
        emailID = extract_emails(text.lower())
        process_emails(emailID)

        row = []
        row.append(personName)
        row.append(contactNum)
        row.append(emailID)
        print(row)
        
        writer.writerow(row)
        break
    f.close()
        

['Krishanu Sen', '+91-9647496690', ['krishanusen.krishanu@gmail.com']]
